# Multiplying upper triangular matrices

This notebook helps you implement the operation $ C := U R $ where $ C, U, R \in \mathbb{R}^{n \times n} $, and $ U $ and $ R $ are upper triangular.  $ U $ and $ R $ are stored in the upper triangular part of numpy matrices <code> U </code> and <code> R </code>. The upper triangular part of matrix <code> C </code> is to be overwritten with the resulting upper triangular matrix.  

First, we create some matrices.

In [2]:
import numpy as np

n = 5

C = np.matrix( np.random.random( (n, n) ) )
print( 'C = ' )
print( C )

Cold = np.matrix( np.zeros( (n,n ) ) )
Cold = np.matrix( np.copy( C ) )           # an alternative way of doing a "hard" copy, in this case of a matrix
    
U = np.matrix( np.random.random( (n, n) ) )
print( 'U = ' )
print( U )
print(type(U) is np.matrix)

R = np.matrix( np.random.random( (n, n) ) )
print( 'R = ' )
print( R )

C = 
[[0.96432275 0.31154227 0.51560493 0.78794521 0.66349128]
 [0.1223971  0.74027042 0.53783587 0.17086467 0.99093872]
 [0.25575934 0.12316098 0.92956345 0.56976788 0.53680743]
 [0.13735046 0.1731707  0.18920311 0.32263394 0.62339423]
 [0.34399602 0.86507926 0.42830015 0.36372519 0.95590157]]
U = 
[[0.19275772 0.36237074 0.39231098 0.24244855 0.53565997]
 [0.84123975 0.19779797 0.45987338 0.24780325 0.69466945]
 [0.63714614 0.75634541 0.98683114 0.26105325 0.83766786]
 [0.17678535 0.39045843 0.10915317 0.30877382 0.12707606]
 [0.25186828 0.93800172 0.6315358  0.78550804 0.34355951]]
True
R = 
[[0.36855951 0.7567906  0.02612422 0.33017425 0.59090497]
 [0.49147603 0.16478768 0.17832443 0.75921587 0.04949801]
 [0.04394857 0.41242228 0.97071787 0.95544661 0.91078616]
 [0.22210409 0.14048244 0.17343268 0.38742541 0.96547108]
 [0.48934138 0.07842986 0.32252504 0.63576143 0.36833411]]


## <h2>The algorithm  </h2>  <image src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/5_5_1_10_Answer.png" alt="Upper triangular matrix-matrix multiplication" width="80%">

<h2> The routine <code> Trtrmm_uu_unb_var1( U, R, C ) </code> </h2>

This routine computes $ C := U R + C $.  The "\_uu\_" means that $ U $ and $ R $ are upper triangular matrices (which means $ C $ is too).  However, the lower triangular parts of numpy matrices <code> U </code>, <code> R </code>, and <code> C </code> are not to be "touched".
    
The specific laff function you will want to use is some subset of
<ul>
<li> <code> laff.gemv( trans, alpha, A, x, beta, y ) </code> which computes $ y := \alpha A x + \beta y $ or $ y := \alpha A^T x + \beta y $ depending on whether <code> trans = 'No transpose' </code> or <code> trans = 'Transpose' </code> </li>
    <li> <code> laff.ger( alpha, x, y, A ) </code> which computes the rank-1 update (adds a multiple of an outer product to a matrix)
$ A := \alpha x y^T + A $. </li>
    <li> <code> laff.axpy( alpha, x, y ) </code></li>
    <li>    <code> laff.dots( x, y, alpha ) </code></li>
</ul>

<font color=red> Hint:</font>  If you multiply with $ U_{00} $, you will want to use <code> np.triu( U00 ) </code> to make sure you don't compute with the nonzeroes below the diagonal.

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [5]:
import laff as laff
import flame

def Trtrmm_uu_unb_var1(U, R, C):

    UTL, UTR, \
    UBL, UBR  = flame.part_2x2(U, \
                               0, 0, 'TL')

    RTL, RTR, \
    RBL, RBR  = flame.part_2x2(R, \
                               0, 0, 'TL')

    CTL, CTR, \
    CBL, CBR  = flame.part_2x2(C, \
                               0, 0, 'TL')

    while UTL.shape[0] < U.shape[0]:

        U00,  u01,       U02,  \
        u10t, upsilon11, u12t, \
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, \
                                                         UBL, UBR, \
                                                         1, 1, 'BR')

        R00,  r01,   R02,  \
        r10t, rho11, r12t, \
        R20,  r21,   R22   = flame.repart_2x2_to_3x3(RTL, RTR, \
                                                     RBL, RBR, \
                                                     1, 1, 'BR')

        C00,  c01,     C02,  \
        c10t, gamma11, c12t, \
        C20,  c21,     C22   = flame.repart_2x2_to_3x3(CTL, CTR, \
                                                       CBL, CBR, \
                                                       1, 1, 'BR')

        #------------------------------------------------------------#

        # imporant:  You do not need to recompute C00 = U00 * R00!!!!
        laff.gemv( 'No transpose', 1.0, np.triu( U00 ), np.matrix(r01), 0.0, c01 )
        laff.axpy( rho11, u01, c01 )
        laff.dot( rho11, upsilon11, gamma11 )

        
        #------------------------------------------------------------#

        UTL, UTR, \
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  \
                                               u10t, upsilon11, u12t, \
                                               U20,  u21,       U22,  \
                                               'TL')

        RTL, RTR, \
        RBL, RBR  = flame.cont_with_3x3_to_2x2(R00,  r01,   R02,  \
                                               r10t, rho11, r12t, \
                                               R20,  r21,   R22,  \
                                               'TL')

        CTL, CTR, \
        CBL, CBR  = flame.cont_with_3x3_to_2x2(C00,  c01,     C02,  \
                                               c10t, gamma11, c12t, \
                                               C20,  c21,     C22,  \
                                               'TL')

    flame.merge_2x2(CTL, CTR, \
                    CBL, CBR, C)





In [6]:
C = np.matrix( np.copy( Cold ) )               # restore C 

Trtrmm_uu_unb_var1( U, R, C )

# compute it using numpy *.  This is a little complex, since we want to make sure we
#   don't change the lower triangular part of C
# Cref = np.tril( Cold, -1 ) + np.triu( U ) * np.triu( R )
Cref = np.triu( U ) * np.triu( R ) + np.tril( Cold, -1 )
print( 'C - Cref' )
print( C - Cref )

AssertionError: laff.gemv: vector x must be a 2D numpy.matrix

In theory, the result matrix should be (approximately) zero.

## Watch the algorithm at work!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.